# 🚨 Smart Safety Alert System

**Welcome to your first intelligent safety system!**

You'll build a **safety alert system** that combines person detection with live motor data to make intelligent decisions about when to sound alerts.

## What You'll Build

- **Person Detection System**: Uses AI to detect people in camera view
- **Motor RPM Monitoring**: Reads live data from VESC motor controller  
- **Smart Alert Logic**: Buzzer sounds when person detected AND motor RPM > 10
- **Simple Configuration**: Easy-to-modify values for students to experiment

## System Configuration (Modify These Values!)

**Change these values to customize your safety system:**

In [ ]:
# 🔧 CONFIGURATION - CHANGE THESE VALUES!

# ========================================
# ALERT TRIGGER CONDITIONS
# ========================================
PERSON_CONFIDENCE_THRESHOLD = 0.70  # Minimum confidence for person detection (try 0.60, 0.75, 0.85)
MIN_ALERT_RPM = 10                  # RPM threshold for motor safety alert (try 5, 15, 25)

# ========================================  
# BUZZER SETTINGS
# ========================================
BUZZER_PIN = 17           # GPIO pin for buzzer (don't change unless rewired)
ALERT_BEEP_COUNT = 3      # Number of beeps per alert (try 2, 4, 5)
BEEP_DURATION = 0.15      # How long each beep lasts (try 0.1, 0.2)
ALERT_COOLDOWN = 2.0      # Seconds between alerts (try 1.0, 3.0, 5.0)

print(f"👤 Person detection threshold: {PERSON_CONFIDENCE_THRESHOLD:.0%}")  
print(f"🚗 Motor RPM alert threshold: {MIN_ALERT_RPM} RPM")
print(f"🔊 Buzzer: {ALERT_BEEP_COUNT} beeps on GPIO {BUZZER_PIN}")
print(f"⏱️  Alert cooldown: {ALERT_COOLDOWN} seconds")

## System Setup

In [ ]:
# 🚀 SYSTEM SETUP - Initialize all components

import sys
import os
import time
import threading
import cv2
import numpy as np
from IPython.display import display, clear_output, Image
import ipywidgets as widgets

# Add paths for camera and VESC
sys.path.append('/home/pi/picamera2')
sys.path.append('/home/pi/RaspberryPi-CAN')

# Import required modules
from picamera2 import MappedArray, Picamera2
from picamera2.devices import IMX500
from picamera2.devices.imx500 import NetworkIntrinsics

try:
    from student_api import VESCStudentAPI
    import RPi.GPIO as GPIO
    GPIO.setwarnings(False)
    print("✅ All imports successful")
except ImportError as e:
    print(f"❌ Import failed: {e}")

# Initialize VESC motor controller
print("🔌 Connecting to VESC...")
vesc_api = VESCStudentAPI()
vesc_controller = None

if vesc_api.start():
    time.sleep(6)
    controllers = vesc_api.get_connected_controllers()
    if controllers:
        vesc_controller = vesc_api.get_controller(controllers[0])
        if vesc_controller.is_connected():
            print(f"✅ VESC connected! Battery: {vesc_controller.get_input_voltage():.1f}V")
        else:
            print("❌ VESC not responding")
    else:
        print("❌ No VESC found")
else:
    print("❌ VESC startup failed")

# Initialize buzzer GPIO
try:
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(BUZZER_PIN, GPIO.OUT, initial=GPIO.LOW)
    print(f"✅ Buzzer ready on GPIO {BUZZER_PIN}")
    buzzer_ready = True
except:
    print("❌ Buzzer setup failed")
    buzzer_ready = False

# Global variables
system_running = False
last_alert_time = 0
alert_count = 0

print()
print("System ready! Run the next cell to start monitoring.")
print(f"Alert triggers: Person ≥{PERSON_CONFIDENCE_THRESHOLD:.0%} + Motor >{MIN_ALERT_RPM} RPM")

## Main Code

In [ ]:
# 🚨 SMART SAFETY ALERT SYSTEM - Auto-starts when you run this cell!

# COCO labels for object detection
labels = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "", "backpack",
    "umbrella", "", "", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard",
    "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
    "tennis racket", "bottle", "", "wine glass", "cup", "fork", "knife", "spoon", "bowl",
    "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza",
    "donut", "cake", "chair", "couch", "potted plant", "bed", "", "dining table", "", "",
    "toilet", "", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave",
    "oven", "toaster", "sink", "refrigerator", "", "book", "clock", "vase", "scissors",
    "teddy bear", "hair drier", "toothbrush"
]

def trigger_alert(person_confidence, motor_rpm):
    """Check if alert should trigger and sound buzzer"""
    global last_alert_time, alert_count
    
    # Check cooldown
    if time.time() - last_alert_time < ALERT_COOLDOWN:
        return False
    
    # Check both conditions: person detected AND motor spinning
    if person_confidence >= PERSON_CONFIDENCE_THRESHOLD and abs(motor_rpm) > MIN_ALERT_RPM:
        alert_count += 1
        last_alert_time = time.time()
        
        # Sound buzzer
        if buzzer_ready:
            for _ in range(ALERT_BEEP_COUNT):
                GPIO.output(BUZZER_PIN, GPIO.HIGH)
                time.sleep(BEEP_DURATION)
                GPIO.output(BUZZER_PIN, GPIO.LOW)
                time.sleep(0.05)
        
        print(f"🚨 ALERT #{alert_count}: Person {person_confidence:.0%}, Motor {motor_rpm:.1f} RPM")
        return True
    return False

def parse_detections(metadata, imx500, intrinsics, picam2):
    """Get detections from AI model"""
    np_outputs = imx500.get_outputs(metadata, add_batch=True)
    if np_outputs is None:
        return []
    
    boxes, scores, classes = np_outputs[0][0], np_outputs[1][0], np_outputs[2][0]
    
    # Convert to proper format
    if intrinsics.bbox_normalization:
        boxes = boxes / imx500.get_input_size()[1]
    if intrinsics.bbox_order == "xy":
        boxes = boxes[:, [1, 0, 3, 2]]
    
    # Create detections
    detections = []
    for box, score, category in zip(boxes, scores, classes):
        if score > 0.5:  # Basic threshold
            coords = imx500.convert_inference_coords(box, metadata, picam2)
            detections.append({
                'category': int(category),
                'confidence': float(score),
                'box': coords
            })
    
    return detections

def draw_detections(request, stream="main"):
    """Draw boxes on camera feed"""
    global current_detections
    if not current_detections:
        return
        
    with MappedArray(request, stream) as m:
        for detection in current_detections:
            x, y, w, h = detection['box']
            conf = detection['confidence']
            obj_name = labels[detection['category']]
            
            # Red box for person, green for others
            color = (255, 0, 0) if obj_name == "person" else (0, 255, 0)
            cv2.rectangle(m.array, (x, y), (x + w, y + h), color, 2)
            
            # Label
            label = f"{obj_name} {conf:.0%}"
            cv2.putText(m.array, label, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

# Create stop button
stop_button = widgets.Button(description='⏹️ STOP SYSTEM', button_style='danger')
output = widgets.Output()
display(widgets.VBox([stop_button, output]))

def stop_system(button):
    global system_running
    system_running = False
    if buzzer_ready:
        GPIO.output(BUZZER_PIN, GPIO.LOW)
    with output:
        print("⏹️ System stopped!")

stop_button.on_click(stop_system)

# Main system loop
current_detections = []
system_running = True

with output:
    print("🚀 STARTING SMART SAFETY SYSTEM")
    print(f"🎯 Alerts: Person ≥{PERSON_CONFIDENCE_THRESHOLD:.0%} + Motor >{MIN_ALERT_RPM} RPM")
    print("-" * 40)

def run_system():
    """Main system function"""
    global current_detections, system_running
    
    try:
        # Initialize camera
        model_path = "/usr/share/imx500-models/imx500_network_ssd_mobilenetv2_fpnlite_320x320_pp.rpk"
        imx500 = IMX500(model_path)
        intrinsics = imx500.network_intrinsics or NetworkIntrinsics()
        intrinsics.task = "object detection"
        intrinsics.labels = labels
        intrinsics.update_with_defaults()
        
        picam2 = Picamera2(imx500.camera_num)
        config = picam2.create_preview_configuration(controls={"FrameRate": 30}, buffer_count=12)
        
        imx500.show_network_fw_progress_bar()
        picam2.start(config, show_preview=False)
        picam2.pre_callback = draw_detections
        
        display_handle = display(None, display_id=True)
        
        frame_count = 0
        while system_running:
            frame_count += 1
            
            # Get AI detections
            current_detections = parse_detections(picam2.capture_metadata(), imx500, intrinsics, picam2)
            
            # Find best person detection
            person_confidence = 0.0
            for detection in current_detections:
                if labels[detection['category']] == "person":
                    person_confidence = max(person_confidence, detection['confidence'])
            
            # Get motor RPM
            motor_rpm = 0.0
            if vesc_controller:
                motor_rpm = vesc_controller.get_rpm() or 0.0
            
            # Check for alert
            if person_confidence > 0:
                trigger_alert(person_confidence, motor_rpm)
            
            # Update display
            request = picam2.capture_request()
            img = request.make_array("main")
            _, jpeg = cv2.imencode('.jpg', img)
            display_handle.update(Image(data=jpeg.tobytes()))
            request.release()
            
            # Status update every 2 seconds
            if frame_count % 60 == 0:
                with output:
                    print(f"Status: {person_confidence:.0%} person, {motor_rpm:.1f} RPM, {alert_count} alerts")
            
            time.sleep(0.033)  # ~30 FPS
            
        picam2.stop()
        
    except Exception as e:
        with output:
            print(f"❌ Error: {e}")
        system_running = False

# Start system in background thread
threading.Thread(target=run_system, daemon=True).start()

print("Stand in front of camera and spin motor to test alerts!")
print("Try modifying the configuration values and restarting!")